In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
%load_ext tensorboard

In [2]:
'''
import glob
import pickle
files = glob.glob("../../datasets/ant-*.csv", recursive = True)
globa = pd.concat(map(pd.read_csv, files))
pickle.dump( globa, open( "ant_master.p", "wb" ) )
'''

'\nimport glob\nimport pickle\nfiles = glob.glob("../../datasets/ant-*.csv", recursive = True)\ngloba = pd.concat(map(pd.read_csv, files))\npickle.dump( globa, open( "ant_master.p", "wb" ) )\n'

In [3]:
ant_master = pickle.load( open( "ant_master.p", "rb" ) )

In [4]:
ant_master

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,ant,1.7,org.apache.tools.ant.taskdefs.rmic.RmicAdapter...,3,1,0,10,18,3,1,...,0.000000,0,0.000000,0.444444,0,0,32.666667,1,0.6667,0
1,ant,1.7,org.apache.tools.ant.taskdefs.optional.perforc...,5,2,0,4,13,0,1,...,1.000000,1,0.700000,0.500000,0,0,13.400000,1,0.6000,0
2,ant,1.7,org.apache.tools.ant.taskdefs.optional.junit.O...,1,2,0,1,3,0,0,...,0.000000,0,1.000000,1.000000,0,0,6.000000,0,0.0000,0
3,ant,1.7,org.apache.tools.ant.taskdefs.optional.perforc...,8,1,9,13,20,12,9,...,0.200000,1,0.000000,0.406250,0,0,11.000000,1,0.8750,0
4,ant,1.7,org.apache.tools.ant.taskdefs.WaitFor,9,3,0,5,26,16,0,...,1.000000,0,0.800000,0.388889,0,0,19.000000,2,1.0000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,ant,1.4,org.apache.tools.ant.TaskAdapter,5,3,0,7,19,4,4,...,0.000000,0,0.902439,0.400000,1,1,28.800000,1,0.8000,0
174,ant,1.4,org.apache.tools.ant.taskdefs.rmic.DefaultRmic...,17,1,3,15,66,90,4,...,1.000000,2,0.000000,0.197917,0,0,35.882353,14,2.4118,0
175,ant,1.4,org.apache.tools.ant.IntrospectionHelper,23,1,0,31,82,201,22,...,0.444444,0,0.000000,0.318182,0,0,45.739130,24,2.3043,0
176,ant,1.4,org.apache.tools.ant.taskdefs.compilers.Defaul...,11,1,8,22,86,15,8,...,1.000000,7,0.000000,0.266667,0,0,77.727273,22,4.6364,1


In [5]:
test = ant_master[ant_master['version']==1.7]
test

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,ant,1.7,org.apache.tools.ant.taskdefs.rmic.RmicAdapter...,3,1,0,10,18,3,1,...,0.000000,0,0.000000,0.444444,0,0,32.666667,1,0.6667,0
1,ant,1.7,org.apache.tools.ant.taskdefs.optional.perforc...,5,2,0,4,13,0,1,...,1.000000,1,0.700000,0.500000,0,0,13.400000,1,0.6000,0
2,ant,1.7,org.apache.tools.ant.taskdefs.optional.junit.O...,1,2,0,1,3,0,0,...,0.000000,0,1.000000,1.000000,0,0,6.000000,0,0.0000,0
3,ant,1.7,org.apache.tools.ant.taskdefs.optional.perforc...,8,1,9,13,20,12,9,...,0.200000,1,0.000000,0.406250,0,0,11.000000,1,0.8750,0
4,ant,1.7,org.apache.tools.ant.taskdefs.WaitFor,9,3,0,5,26,16,0,...,1.000000,0,0.800000,0.388889,0,0,19.000000,2,1.0000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
740,ant,1.7,org.apache.tools.ant.taskdefs.Javadoc,92,3,0,34,261,3726,8,...,0.970588,11,0.291339,0.112476,2,2,28.021739,15,1.5543,4
741,ant,1.7,org.apache.tools.ant.types.selectors.BaseSelector,6,3,6,10,10,3,7,...,1.000000,0,0.857143,0.500000,0,0,6.500000,3,1.5000,0
742,ant,1.7,org.apache.tools.ant.types.resources.selectors...,7,3,5,9,26,0,5,...,1.000000,0,0.857143,0.314286,1,3,19.000000,3,1.5714,0
743,ant,1.7,org.apache.tools.ant.taskdefs.compilers.Gcj,5,2,0,8,34,8,1,...,1.000000,0,0.884615,1.000000,0,0,42.000000,11,3.4000,1


In [6]:
train = ant_master[ant_master['version']!=1.7]
train

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,ant,1.6,org.apache.tools.ant.input.PropertyFileInputHa...,3,1,0,3,19,0,0,...,0.500000,0,0.000000,0.666667,0,0,33.000000,1,0.6667,0
1,ant,1.6,org.apache.tools.ant.types.selectors.OrSelector,3,5,0,6,14,3,5,...,0.000000,0,0.968750,0.555556,1,1,16.333333,3,1.6667,0
2,ant,1.6,org.apache.tools.ant.filters.util.ChainReaderH...,10,1,0,15,39,19,4,...,0.250000,1,0.000000,0.228571,0,0,32.600000,3,1.1000,0
3,ant,1.6,org.apache.tools.ant.taskdefs.Ant,24,3,0,17,118,72,2,...,0.916667,1,0.616667,0.185185,3,4,42.916667,6,1.7083,2
4,ant,1.6,org.apache.tools.ant.taskdefs.JikesOutputParser,12,1,0,3,22,44,1,...,1.000000,1,0.000000,0.261905,0,0,11.416667,4,1.2500,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,ant,1.4,org.apache.tools.ant.TaskAdapter,5,3,0,7,19,4,4,...,0.000000,0,0.902439,0.400000,1,1,28.800000,1,0.8000,0
174,ant,1.4,org.apache.tools.ant.taskdefs.rmic.DefaultRmic...,17,1,3,15,66,90,4,...,1.000000,2,0.000000,0.197917,0,0,35.882353,14,2.4118,0
175,ant,1.4,org.apache.tools.ant.IntrospectionHelper,23,1,0,31,82,201,22,...,0.444444,0,0.000000,0.318182,0,0,45.739130,24,2.3043,0
176,ant,1.4,org.apache.tools.ant.taskdefs.compilers.Defaul...,11,1,8,22,86,15,8,...,1.000000,7,0.000000,0.266667,0,0,77.727273,22,4.6364,1


In [7]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [8]:
x_train

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
0,3,1,0,3,19,0,0,3,2,0.750000,104,0.500000,0,0.000000,0.666667,0,0,33.000000,1,0.6667
1,3,5,0,6,14,3,5,2,3,2.000000,52,0.000000,0,0.968750,0.555556,1,1,16.333333,3,1.6667
2,10,1,0,15,39,19,4,12,8,0.888889,344,0.250000,1,0.000000,0.228571,0,0,32.600000,3,1.1000
3,24,3,0,17,118,72,2,15,17,0.851449,1066,0.916667,1,0.616667,0.185185,3,4,42.916667,6,1.7083
4,12,1,0,3,22,44,1,2,5,0.805195,156,1.000000,1,0.000000,0.261905,0,0,11.416667,4,1.2500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,5,3,0,7,19,4,4,4,4,0.750000,151,0.000000,0,0.902439,0.400000,1,1,28.800000,1,0.8000
174,17,1,3,15,66,90,4,12,6,0.791667,630,1.000000,2,0.000000,0.197917,0,0,35.882353,14,2.4118
175,23,1,0,31,82,201,22,27,17,0.893939,1093,0.444444,0,0.000000,0.318182,0,0,45.739130,24,2.3043
176,11,1,8,22,86,15,8,14,4,0.728571,887,1.000000,7,0.000000,0.266667,0,0,77.727273,22,4.6364


In [9]:
x_test

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
0,3,1,0,10,18,3,1,9,1,1.100000,106,0.000000,0,0.000000,0.444444,0,0,32.666667,1,0.6667
1,5,2,0,4,13,0,1,4,4,0.625000,76,1.000000,1,0.700000,0.500000,0,0,13.400000,1,0.6000
2,1,2,0,1,3,0,0,1,1,2.000000,7,0.000000,0,1.000000,1.000000,0,0,6.000000,0,0.0000
3,8,1,9,13,20,12,9,4,8,0.800000,101,0.200000,1,0.000000,0.406250,0,0,11.000000,1,0.8750
4,9,3,0,5,26,16,0,5,7,0.750000,185,1.000000,0,0.800000,0.388889,0,0,19.000000,2,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
740,92,3,0,34,261,3726,8,34,81,0.960246,2704,0.970588,11,0.291339,0.112476,2,2,28.021739,15,1.5543
741,6,3,6,10,10,3,7,3,6,0.400000,46,1.000000,0,0.857143,0.500000,0,0,6.500000,3,1.5000
742,7,3,5,9,26,0,5,4,6,0.000000,141,1.000000,0,0.857143,0.314286,1,3,19.000000,3,1.5714
743,5,2,0,8,34,8,1,7,3,0.500000,216,1.000000,0,0.884615,1.000000,0,0,42.000000,11,3.4000


In [10]:
y_train = train['bug']
y_test = test['bug']

In [11]:
y_train

0      0
1      0
2      0
3      2
4      0
      ..
173    0
174    0
175    0
176    1
177    0
Name: bug, Length: 947, dtype: int64

In [12]:
y_test

0      0
1      0
2      0
3      0
4      1
      ..
740    4
741    0
742    0
743    1
744    0
Name: bug, Length: 745, dtype: int64

In [13]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [14]:
x_train_np

array([[ 3.        ,  1.        ,  0.        , ..., 33.        ,
         1.        ,  0.6667    ],
       [ 3.        ,  5.        ,  0.        , ..., 16.33333333,
         3.        ,  1.6667    ],
       [10.        ,  1.        ,  0.        , ..., 32.6       ,
         3.        ,  1.1       ],
       ...,
       [23.        ,  1.        ,  0.        , ..., 45.73913043,
        24.        ,  2.3043    ],
       [11.        ,  1.        ,  8.        , ..., 77.72727273,
        22.        ,  4.6364    ],
       [ 4.        ,  2.        ,  0.        , ..., 13.5       ,
         5.        ,  1.75      ]])

In [15]:
x_test_np

array([[ 3.        ,  1.        ,  0.        , ..., 32.66666667,
         1.        ,  0.6667    ],
       [ 5.        ,  2.        ,  0.        , ..., 13.4       ,
         1.        ,  0.6       ],
       [ 1.        ,  2.        ,  0.        , ...,  6.        ,
         0.        ,  0.        ],
       ...,
       [ 7.        ,  3.        ,  5.        , ..., 19.        ,
         3.        ,  1.5714    ],
       [ 5.        ,  2.        ,  0.        , ..., 42.        ,
        11.        ,  3.4       ],
       [ 2.        ,  1.        ,  0.        , ..., 40.        ,
         6.        ,  3.        ]])

In [16]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [17]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [18]:
train_x

array([[[ 3.        ,  1.        ,  0.        , ..., 33.        ,
          1.        ,  0.6667    ]],

       [[ 3.        ,  5.        ,  0.        , ..., 16.33333333,
          3.        ,  1.6667    ]],

       [[10.        ,  1.        ,  0.        , ..., 32.6       ,
          3.        ,  1.1       ]],

       ...,

       [[23.        ,  1.        ,  0.        , ..., 45.73913043,
         24.        ,  2.3043    ]],

       [[11.        ,  1.        ,  8.        , ..., 77.72727273,
         22.        ,  4.6364    ]],

       [[ 4.        ,  2.        ,  0.        , ..., 13.5       ,
          5.        ,  1.75      ]]])

In [19]:
test_y

array([ 0,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  1,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,
        1,  2,  0,  1,  1,  0,  3,  0,  0,  0,  0,  1,  6,  0,  0,  0,  1,
        0,  0,  4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,
        0,  0,  0,  1,  0,  3,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  2,
        0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,
        0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  1,  0,  0,  1,  0,  0,  0,  0,  0,  2,  0,  0,  1,
        0,  8,  1,  0,  0,  0,  3,  2,  0,  0,  0,  0,  0,  1,  0,  0,  0,
        0,  0,  1,  0,  4,  0,  0,  0,  0,  0,  0,  0,  1,  1,  0,  0,  0,
        0,  0,  0,  0,  0,  3,  2,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,
        2,  0,  0,  3,  0,  0,  0,  2,  0,  1,  1,  0,  4,  0,  0,  0,  1,
        1,  0,  0,  2,  0

In [2]:
# Designing and initializing the model.
from keras.layers import LSTM, Dense, SimpleRNN
# Designing and initializing the model.
model = Sequential()
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50, dropout = 0.2, return_sequences=True))
model.add(LSTM(20, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [21]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, batch_size = 10, callbacks=[tensorboard_callback])

Epoch 1/100
95/95 [==============================] - 5s 7ms/step - loss: 0.7668 - mse: 0.7668 - mae: 0.3424 - root_mean_squared_error: 0.8756 - mean_squared_logarithmic_error: 0.1640
Epoch 2/100
95/95 [==============================] - 0s 4ms/step - loss: 0.7020 - mse: 0.7020 - mae: 0.4203 - root_mean_squared_error: 0.8379 - mean_squared_logarithmic_error: 0.1446
Epoch 3/100
95/95 [==============================] - 0s 5ms/step - loss: 0.6605 - mse: 0.6605 - mae: 0.4528 - root_mean_squared_error: 0.8127 - mean_squared_logarithmic_error: 0.1408
Epoch 4/100
95/95 [==============================] - 0s 4ms/step - loss: 0.6277 - mse: 0.6277 - mae: 0.4250 - root_mean_squared_error: 0.7923 - mean_squared_logarithmic_error: 0.1314
Epoch 5/100
95/95 [==============================] - 0s 5ms/step - loss: 0.6018 - mse: 0.6018 - mae: 0.4052 - root_mean_squared_error: 0.7758 - mean_squared_logarithmic_error: 0.1272
Epoch 6/100
95/95 [==============================] - 0s 4ms/step - loss: 0.5910 - mse

95/95 [==============================] - 0s 5ms/step - loss: 0.3549 - mse: 0.3549 - mae: 0.3141 - root_mean_squared_error: 0.5957 - mean_squared_logarithmic_error: 0.0876
Epoch 89/100
95/95 [==============================] - 0s 5ms/step - loss: 0.3560 - mse: 0.3560 - mae: 0.3034 - root_mean_squared_error: 0.5966 - mean_squared_logarithmic_error: 0.0862
Epoch 90/100
95/95 [==============================] - 0s 5ms/step - loss: 0.3512 - mse: 0.3512 - mae: 0.3094 - root_mean_squared_error: 0.5926 - mean_squared_logarithmic_error: 0.0887
Epoch 91/100
95/95 [==============================] - 1s 6ms/step - loss: 0.3482 - mse: 0.3482 - mae: 0.3048 - root_mean_squared_error: 0.5901 - mean_squared_logarithmic_error: 0.0864
Epoch 92/100
95/95 [==============================] - 1s 6ms/step - loss: 0.3594 - mse: 0.3594 - mae: 0.3180 - root_mean_squared_error: 0.5995 - mean_squared_logarithmic_error: 0.0891
Epoch 93/100
95/95 [==============================] - 1s 6ms/step - loss: 0.3612 - mse: 0.361

In [22]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

In [23]:
score = model.evaluate(test_x, test_y)
score

24/24 [==============================] - 1s 2ms/step - loss: 1.0063 - mse: 1.0063 - mae: 0.4779 - root_mean_squared_error: 1.0032 - mean_squared_logarithmic_error: 0.1755


[1.0063186883926392,
 1.0063186883926392,
 0.4778525233268738,
 1.0031543970108032,
 0.17551973462104797]